In [1]:
from utils.read_file import read_target_insurance_pdf, read_json

In [24]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"
FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"

In [8]:
question = read_json(QUESTION_PATH)["questions"]

{'qid': 1,
 'source': [442, 115, 440, 196, 431, 392, 14, 51],
 'query': '匯款銀行及中間行所收取之相關費用由誰負擔?',
 'category': 'insurance'}

In [11]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer('thenlper/gte-large-zh')

c:\Users\weiso131\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_ans(embedded_model, question_data: dict, documents: list):
    document_ebeddings = embedded_model.encode(documents)

    # 使用者查詢
    user_query = question_data["query"]

    # 查詢文本轉換成嵌入向量
    query_embedding = embbeded_model.encode([user_query])

    # 計算相似度
    similarities = cosine_similarity(query_embedding, document_ebeddings)
    
    for i in range(len(similarities[0])):
        print(f"{question_data["source"][i]} : {similarities[0][i]}")


    return question_data["source"][np.argmax(similarities)]

In [28]:
faq = read_json(FAQ_PATH)
def generate_faq_retrieve(faq: dict, id: int) -> str:

    text = ""

    for faq_data in faq[str(id)]:

        question = faq_data["question"]
        answers = faq_data["answers"]
        text += f"問題:{question}, 回答:{answers}\n"
    return text

def read_target_faq(faq: dict, indexs: list) -> list:

    all_texts = []

    for i in indexs:
        all_texts.append(generate_faq_retrieve(faq, i))
    
    return all_texts

In [30]:

document = read_target_faq(faq, question[100]["source"])

get_ans(embbeded_model, question[100], document)

69 : 0.3763333559036255
398 : 0.4597470164299011
229 : 0.3988722562789917
474 : 0.46297940611839294
382 : 0.38264065980911255
207 : 0.3546384871006012
377 : 0.32410815358161926
104 : 0.3161534070968628
32 : 0.4784950017929077
148 : 0.4518640637397766
476 : 0.25867533683776855
238 : 0.3116232752799988
441 : 0.43779969215393066
233 : 0.3853197693824768
558 : 0.8296957015991211


558